In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
import numpy.random
import numpy.linalg
import scipy.io
import scipy.stats
import sklearn.metrics

inNotebook = True # change this to True if you use a notebook
def nextplot():
    if inNotebook:
        plt.figure()  # this creates a new plot
    else:
        plt.clf()     # and this clears the current one

# Load the data

In [ ]:
data = scipy.io.loadmat('/tmp/spamData.mat')
X = data['Xtrain']
N = X.shape[0]
D = X.shape[1]
Xtest = data['Xtest']
Ntest = Xtest.shape[0]
y = data['ytrain'].squeeze().astype(int)
ytest = data['ytest'].squeeze().astype(int)

features = np.array([
    "word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d",
    "word_freq_our", "word_freq_over", "word_freq_remove", "word_freq_internet",
    "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will",
    "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free",
    "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit",
    "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money",
    "word_freq_hp", "word_freq_hpl", "word_freq_george", "word_freq_650",
    "word_freq_lab", "word_freq_labs", "word_freq_telnet", "word_freq_857",
    "word_freq_data", "word_freq_415", "word_freq_85", "word_freq_technology",
    "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct",
    "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project",
    "word_freq_re", "word_freq_edu", "word_freq_table", "word_freq_conference",
    "char_freq_;", "char_freq_(", "char_freq_[", "char_freq_!",
    "char_freq_$", "char_freq_#", "capital_run_length_average", "capital_run_length_longest",
    "capital_run_length_total" ])

# 1. Dataset Statistics

In [ ]:
# look some dataset statistics
scipy.stats.describe(X)

In [ ]:
# plot the distribution of all features
nextplot()
densities = [ scipy.stats.gaussian_kde(X[:,j]) for j in range(D) ]
xs = np.linspace(0,np.max(X),200)
for j in range(D):
    plt.plot(xs, densities[j](xs), label=j)
plt.legend(ncol=5)

In [ ]:
# this plots is not really helpful; go now explore further
# YOUR CODE HERE

# Try different bandwidths and scales

# [0, 0.3]
nextplot()
densities = [ scipy.stats.gaussian_kde(X[:,j]) for j in range(D) ]
xs = np.linspace(0, 0.3,200)
for j in range(D):
    plt.plot(xs, densities[j](xs), label=j)
plt.legend(ncol=5)

# Silverman
nextplot()
densities = [ scipy.stats.gaussian_kde(X[:,j], bw_method='silverman') for j in range(D) ]
xs = np.linspace(0, 0.3,200)
for j in range(D):
    plt.plot(xs, densities[j](xs), label=j)
plt.legend(ncol=5)

# bw_method=0.01
nextplot()
densities = [ scipy.stats.gaussian_kde(X[:,j], bw_method=0.01) for j in range(D) ]
xs = np.linspace(0, 0.3,200)
for j in range(D):
    plt.plot(xs, densities[j](xs), label=j)
plt.legend(ncol=5)

# bw_method=0.5
nextplot()
densities = [ scipy.stats.gaussian_kde(X[:,j]) for j in range(D) ]
xs = np.linspace(0, 0.3,200)
for j in range(D):
    plt.plot(xs, densities[j](xs), label=j)
plt.legend(ncol=5)

In [ ]:
# Let's compute z-scores; create two new variables Xz and Xtestz.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

mean = scaler.mean_
var = scaler.var_

Xz = scaler.transform(X)
Xtestz = scaler.transform(Xtest)

In [ ]:
# Let's check. Xz and Xtestz refer to the normalized datasets just created. We
# will use them throughout.
np.mean(Xz, axis=0)                       # should be all 0
np.var(Xz, axis=0)                        # should be all 1
np.mean(Xtestz, axis=0)                   # what do you get here?
np.var(Xtestz, axis=0)

np.sum(Xz**3)                             # should be: 1925261.15

In [ ]:
# Explore the normalized data
# YOUR CODE HERE

# Show some statistics
scipy.stats.describe(Xz)

# Plot the kde
nextplot()
densities = [ scipy.stats.gaussian_kde(Xz[:,j]) for j in range(D) ]
xs = np.linspace(- np.median(var) * 6,np.median(var) * 6,200)
for j in range(D):
    plt.plot(xs, densities[j](xs), label=j)
plt.legend(ncol=5)

# 2. Maximum Likelihood Estimation

## Helper functions

In [ ]:
def logsumexp(x):
    """Computes log(sum(exp(x)).

    Uses offset trick to reduce risk of numeric over- or underflow. When x is a
    1D ndarray, computes logsumexp of its entries. When x is a 2D ndarray,
    computes logsumexp of each row.

    Keyword arguments:
    x : a 1D or 2D ndarray
    """
    offset = np.max(x, axis=0)
    return offset + np.log(np.sum(np.exp(x-offset), axis=0))

In [ ]:
# Define the logistic function. Make sure it operates on both scalars
# and vectors.

def sigma(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
# this should give:
# [0.5, array([0.26894142, 0.5, 0.73105858])]
[ sigma(0), sigma(np.array([-1,0,1])) ]

In [ ]:
# Define the logarithm of the logistic function. Make sure it operates on both
# scalars and vectors. Perhaps helpful: isinstance(x, np.ndarray).
def logsigma (x):
    if isinstance(x, np.ndarray):
        return np.vectorize(np.log)(sigma(x))
    else:
        return np.log(sigma(x))

In [ ]:
# this should give:
# [-0.69314718055994529, array([-1.31326169, -0.69314718, -0.31326169])]
[ logsigma(0), logsigma(np.array([-1,0,1])) ]

## 2b Log-likelihood and gradient

In [ ]:
def replacer(x):
    if x == 1.0:
        return 0.999
    else:
        return x

def l(y, X, w):
    # https://datascience.stackexchange.com/questions/9302/the-cross-entropy-error-function-in-neural-networks
    
    """Log-likelihood of the logistic regression model.

    Parameters
    ----------
    y : ndarray of shape (N,)
        Binary labels (either 0 or 1).
    X : ndarray of shape (N,D)
        Design matrix.
    w : ndarray of shape (D,)
        Weight vector.
    """
    # YOUR CODE HERE
    p = sigma(X @ w)
    y_1 = y @ np.log(p)
    # This may not be exactly the correct answer, but avoids numerical instability in sigma(X @ w)
    y_0 = (1 - y) @ (1 - np.log(1 - np.vectorize(replacer)(p)))      
    return y_1 + y_0

In [ ]:
# this should give:
# -47066.641667825766
l(y, Xz, np.linspace(-5,5,D))

In [ ]:
def dl(y,X,w):
    """Gradient of the log-likelihood of the logistic regression model.

    Parameters
    ----------
    y : ndarray of shape (N,)
        Binary labels (either 0 or 1).
    X : ndarray of shape (N,D)
        Design matrix.
    w : ndarray of shape (D,)
        Weight vector.

    Returns
    -------
    ndarray of shape (D,)
    """
    # YOUR CODE HERE
    return -(sigma(X @ w) - y) @ X

In [ ]:
# this should give:
# array([  551.33985842,   143.84116318,   841.83373606,   156.87237578,
#          802.61217579,   795.96202907,   920.69045803,   621.96516752,
#          659.18724769,   470.81259805,   771.32406968,   352.40325626,
#          455.66972482,   234.36600888,   562.45454038,   864.83981264,
#          787.19723703,   649.48042176,   902.6478154 ,   544.00539886,
#         1174.78638035,   120.3598967 ,   839.61141672,   633.30453444,
#         -706.66815087,  -630.2039816 ,  -569.3451386 ,  -527.50996698,
#         -359.53701083,  -476.64334832,  -411.60620464,  -375.11950586,
#         -345.37195689,  -376.22044258,  -407.31761977,  -456.23251936,
#         -596.86960184,  -107.97072355,  -394.82170044,  -229.18125598,
#         -288.46356547,  -362.13402385,  -450.87896465,  -277.03932676,
#         -414.99293368,  -452.28771693,  -167.54649092,  -270.9043748 ,
#         -252.20140951,  -357.72497343,  -259.12468742,   418.35938483,
#          604.54173228,    43.10390907,   152.24258478,   378.16731033,
#          416.12032881])
dl(y, Xz, np.linspace(-5,5,D))

## 2c Gradient descent

In [ ]:
# you don't need to modify this function
def optimize(obj_up, theta0, nepochs=50, eps0=0.01):
    """Iteratively minimize a function.

    We use it here to run either gradient descent or stochastic gradient
    descent, using arbitrarly optimization criteria.

    Parameters
    ----------
    obj_up  : a tuple of form (f, update) containing two functions f and update.
              f(theta) computes the value of the objective function.
              update(theta,eps) performs a parameter update with step size eps
              and returns the result.
    theta0  : ndarray of shape (D,)
              Initial parameter vector.
    nepochs : int
              How many epochs (calls to update) to run.
    eps0    : float
              Initial step size.

    Returns
    -------
    A triple consisting of the fitted parameter vector, the values of the
    objective function after every epoch, and the step sizes that were used.
    """

    f, update = obj_up

    # initialize results
    theta = theta0
    values = np.zeros(nepochs+1)
    eps = np.zeros(nepochs+1)
    values[0] = f(theta0)
    eps[0] = eps0

    # now run the update function nepochs times
    for epoch in range(nepochs):
        print("Epoch {:3d}: f={:10.3f}, eps={:10.9f}".format(epoch, values[epoch], eps[epoch]))
        theta = update(theta, eps[epoch])

        # we use the bold driver heuristic
        values[epoch+1] = f(theta)
        if (values[epoch] < values[epoch+1]):
            eps[epoch+1] = eps[epoch]/2.
        else:
            eps[epoch+1] = eps[epoch]*1.05

    # all done
    print("Result after {} epochs: f={}".format(nepochs, values[-1]))
    return theta, values, eps

In [ ]:
# define the objective and update function for one gradient-descent epoch for
# fitting an MLE estimate of logistic regression with gradient descent (should
# return a tuple of two functions; see optimize)
def gd(y,X):
    # YOUR CODE HERE
    def f(w):
        return -l(y, X, w)
    def update(w, a):
        return w + a * dl(y, X, w)
    return [f, update]

In [ ]:
# this should give
# [47066.641667825766,
#  array([  4.13777838e+01,  -1.56745627e+01,   5.75882538e+01,
#           1.14225143e+01,   5.54249703e+01,   5.99229049e+01,
#           7.11220141e+01,   4.84761728e+01,   5.78067289e+01,
#           4.54794720e+01,   7.14638492e+01,   1.51369386e+01,
#           3.36375739e+01,   2.15061217e+01,   5.78014255e+01,
#           6.72743066e+01,   7.00829312e+01,   5.29328088e+01,
#           6.16042473e+01,   5.50018510e+01,   8.94624817e+01,
#           2.74784480e+01,   8.51763599e+01,   5.60363965e+01,
#          -2.55865589e+01,  -1.53788213e+01,  -4.67015412e+01,
#          -2.50356570e+00,  -3.85357592e+00,  -2.21819155e+00,
#           3.32098671e+00,   3.86933390e+00,  -2.00309898e+01,
#           3.84684492e+00,  -2.19847927e-01,  -1.29775457e+00,
#          -1.28374302e+01,  -2.78303173e+00,  -5.61671182e+00,
#           1.73657121e+01,  -6.81197570e+00,  -1.20249002e+01,
#           2.65789491e+00,  -1.39557852e+01,  -2.01135653e+01,
#          -2.72134051e+01,  -9.45952961e-01,  -1.02239111e+01,
#           1.52794293e-04,  -5.18938123e-01,  -3.19717561e+00,
#           4.62953437e+01,   7.87893022e+01,   1.88618651e+01,
#           2.85195027e+01,   5.04698358e+01,   6.41240689e+01])
f, update = gd(y, Xz)
[ f(np.linspace(-5,5,D)),
  update(np.linspace(-5,-5,D), .1) ]

In [ ]:
# you can run gradient descent!
numpy.random.seed(0)
w0 = np.random.normal(size=D)
wz_gd, vz_gd, ez_gd = optimize(gd(y,Xz), w0, nepochs=500)

In [ ]:
# look at how gradient descent made progess
# YOUR CODE HERE
nextplot()
xs = np.linspace(0,100,vz_gd.size)
for j in range(D):
    plt.plot(xs, vz_gd, label=j, ls=':')
    
plt.xlabel('Epoch')
plt.ylabel('Error')

## 2d Stochastic gradient descent

In [ ]:
def sgdepoch(y,X,w,eps):
    """Run one SGD epoch and return the updated weight vector. """
    # Run N stochastic gradient steps (without replacement). Do not rescale each
    # step by factor N (i.e., proceed differently than in the lecture slides).
    # YOUR CODE HERE
    xs = np.array_split(X, N)
    ys = np.array_split(y, N)
    d = w    
    for i in range(N):
        d = d + 0.5 * eps * dl(ys[i], xs[i], w)
    return d

In [ ]:
# when you run this multiple times, with 50% probability you should get the
# following result (there is one other result which is very close):
# array([ -3.43689655e+02,  -1.71161311e+02,  -5.71093536e+02,
#         -5.16478220e+01,   4.66294348e+02,  -3.71589878e+02,
#          5.21493183e+02,   1.25699230e+03,   8.33804130e+02,
#          5.63185399e+02,   1.32761302e+03,  -2.64104011e+02,
#          7.10693307e+02,  -1.75497331e+02,  -1.94174427e+02,
#          1.11641507e+02,  -3.30817509e+02,  -3.46754913e+02,
#          8.48722111e+02,  -1.89136304e+02,  -4.25693844e+02,
#         -1.23084189e+02,  -2.95894797e+02,  -2.35789333e+02,
#         -3.38695243e+02,  -3.05642830e+02,  -2.28975383e+02,
#         -2.38075137e+02,  -1.66702530e+02,  -2.27341599e+02,
#         -1.77575620e+02,  -1.49093855e+02,  -1.70028859e+02,
#         -1.50243833e+02,  -1.82986008e+02,  -2.41143708e+02,
#         -3.31047159e+02,  -5.79991185e+01,  -1.98477863e+02,
#         -1.91264948e+02,  -1.17371919e+02,  -1.66953779e+02,
#         -2.01472565e+02,  -1.23330949e+02,  -3.00857740e+02,
#         -1.95853348e+02,  -7.44868073e+01,  -1.11172370e+02,
#         -1.57618226e+02,  -1.25729512e+00,  -1.45536466e+02,
#         -1.43362438e+02,  -3.00429708e+02,  -9.84391082e+01,
#         -4.54152047e+01,  -5.26492232e+01,  -1.45175427e+02])
sgdepoch(y[1:3],Xz[1:3,:],np.linspace(-5,5,D),1000)

In [ ]:
# define the objective and update function for one gradient-descent epoch for
# fitting an MLE estimate of logistic regression with stochastic gradient descent
# (should return a tuple of two functions; see optimize)
def sgd(y,X):
    # YOUR CODE HERE
    def f(w):
        return -l(y, X, w)
    def update(w, eps):
        return sgdepoch(y, X, w, eps)
    return [f, update]

In [ ]:
# with 50% probability, you should get:
# [40.864973045695081,
#  array([ -3.43689655e+02,  -1.71161311e+02,  -5.71093536e+02,
#          -5.16478220e+01,   4.66294348e+02,  -3.71589878e+02,
#           5.21493183e+02,   1.25699230e+03,   8.33804130e+02,
#           5.63185399e+02,   1.32761302e+03,  -2.64104011e+02,
#           7.10693307e+02,  -1.75497331e+02,  -1.94174427e+02,
#           1.11641507e+02,  -3.30817509e+02,  -3.46754913e+02,
#           8.48722111e+02,  -1.89136304e+02,  -4.25693844e+02,
#          -1.23084189e+02,  -2.95894797e+02,  -2.35789333e+02,
#          -3.38695243e+02,  -3.05642830e+02,  -2.28975383e+02,
#          -2.38075137e+02,  -1.66702530e+02,  -2.27341599e+02,
#          -1.77575620e+02,  -1.49093855e+02,  -1.70028859e+02,
#          -1.50243833e+02,  -1.82986008e+02,  -2.41143708e+02,
#          -3.31047159e+02,  -5.79991185e+01,  -1.98477863e+02,
#          -1.91264948e+02,  -1.17371919e+02,  -1.66953779e+02,
#          -2.01472565e+02,  -1.23330949e+02,  -3.00857740e+02,
#          -1.95853348e+02,  -7.44868073e+01,  -1.11172370e+02,
#          -1.57618226e+02,  -1.25729512e+00,  -1.45536466e+02,
#          -1.43362438e+02,  -3.00429708e+02,  -9.84391082e+01,
#          -4.54152047e+01,  -5.26492232e+01,  -1.45175427e+02])]
f, update = sgd(y[1:3], Xz[1:3,:])
[ f(np.linspace(-5,5,D)),
  update(np.linspace(-5,5,D), 1000) ]

In [ ]:
# you can run stochastic gradient descent!
wz_sgd, vz_sgd, ez_sgd = optimize(sgd(y,Xz), w0, nepochs=500)

## 2e Compare GD and SGD

In [ ]:
# YOUR CODE HERE

# look at how both made progress
nextplot()
xs = np.linspace(0,100,vz_gd.size)
plt.plot(xs, vz_gd, label='gd', ls=':')
    
xs = np.linspace(0,100,vz_sgd.size)
plt.plot(xs, vz_sgd, label='sgd', ls='--')
    
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()

# 3 Prediction

In [ ]:
def predict(Xtest, w):
    """Returns vector of predicted confidence values for logistic regression with
weight vector w."""
    # YOUR CODE HERE
    return sigma(Xtest @ w)
    
def classify(Xtest, w):
    """Returns 0/1 vector of predicted class labels for logistic regression with
weight vector w."""
    # YOUR CODE HERE
    def predictor(x):
        return 1 if x > 0.5 else 0
    return np.vectorize(predictor)(predict(Xtest, w))

In [ ]:
print('\nGradient descent statistics')
print(scipy.stats.describe(wz_gd))

print('\nStochastic gradient descent statistics')
print(scipy.stats.describe(wz_sgd))

# Commented out for better readability of the notebook
# print('\nParameters of gradient descent')
# for j in range(wz_gd.size):
#    print(str(j) + '  ' + str(wz_gd[j]) + '  ' + features[j])

# print('\nParameters of stochastic gradient descent')
# for j in range(wz_sgd.size):
#     print(str(j) + '  ' + str(wz_sgd[j]) + '  ' + features[j])

# Example: confusion matrix
yhat = predict(Xtestz, wz_gd)
ypred = classify(Xtestz, wz_gd)
print('\nGradient descent prediction accuracy\n')
print( sklearn.metrics.classification_report(ytest, ypred) )

# Print sgd confusion matrix
syhat = predict(Xtestz, wz_sgd)
sypred = classify(Xtestz, wz_sgd)
print('\nStochastic gradient descent prediction accuracy\n')
print( sklearn.metrics.classification_report(ytest, sypred) )

In [ ]:
# Example: precision-recall curve (with annotated thresholds)
nextplot()
precision, recall, thresholds = sklearn.metrics.precision_recall_curve(ytest, yhat)
plt.plot(recall, precision)
for x in np.linspace(0,1,10,endpoint=False):
    index = int(x * (precision.size-1))
    plt.text(recall[index], precision[index], "{:3.2f}".format(thresholds[index]))
plt.xlabel("Recall")
plt.ylabel("Precision")

# 4 Maximum Aposteriori Estimation

## 4a Gradient Descent

In [ ]:
def l_l2(y,X,w,lambda_):
    """Log-density of posterior of logistic regression with weights w and L2
regularization parameter lambda_"""
    # YOUR CODE HERE
    return l(y, X, w) + lambda_ * np.sum(w @ w)

In [ ]:
# this should give:
# [-47066.641667825766, -47312.623810682911]
# Follow up mistake from a wrong log-likelihood function. Logic should be correct
[ l_l2(y, Xz, np.linspace(-5,5,D),0), l_l2(y, Xz, np.linspace(-5,5,D),1) ]

In [ ]:
def dl_l2(y,X,w,lambda_):
    """Gradient of log-density of posterior of logistic regression with weights w
and L2 regularization parameter lambda_."""
    # YOUR CODE HERE
    return dl(y, X, w) - lambda_ * w

In [ ]:
# this should give:
# [array([  551.33985842,   143.84116318,   841.83373606,   156.87237578,
#           802.61217579,   795.96202907,   920.69045803,   621.96516752,
#           659.18724769,   470.81259805,   771.32406968,   352.40325626,
#           455.66972482,   234.36600888,   562.45454038,   864.83981264,
#           787.19723703,   649.48042176,   902.6478154 ,   544.00539886,
#          1174.78638035,   120.3598967 ,   839.61141672,   633.30453444,
#          -706.66815087,  -630.2039816 ,  -569.3451386 ,  -527.50996698,
#          -359.53701083,  -476.64334832,  -411.60620464,  -375.11950586,
#          -345.37195689,  -376.22044258,  -407.31761977,  -456.23251936,
#          -596.86960184,  -107.97072355,  -394.82170044,  -229.18125598,
#          -288.46356547,  -362.13402385,  -450.87896465,  -277.03932676,
#          -414.99293368,  -452.28771693,  -167.54649092,  -270.9043748 ,
#          -252.20140951,  -357.72497343,  -259.12468742,   418.35938483,
#           604.54173228,    43.10390907,   152.24258478,   378.16731033,
#           416.12032881]),
#  array([  556.33985842,   148.66259175,   846.4765932 ,   161.33666149,
#           806.89789007,   800.06917193,   924.61902946,   625.71516752,
#           662.75867626,   474.20545519,   774.5383554 ,   355.43897054,
#           458.52686767,   237.04458031,   564.95454038,   867.16124121,
#           789.34009417,   651.44470748,   904.43352968,   545.61254171,
#          1176.21495178,   121.6098967 ,   840.68284529,   634.19739158,
#          -705.95386516,  -629.66826731,  -568.98799574,  -527.33139555,
#          -359.53701083,  -476.82191975,  -411.9633475 ,  -375.65522015,
#          -346.08624261,  -377.11329972,  -408.38904835,  -457.48251936,
#          -598.29817327,  -109.57786641,  -396.60741472,  -231.14554169,
#          -290.60642261,  -364.45545242,  -453.37896465,  -279.71789819,
#          -417.85007654,  -455.32343122,  -170.76077664,  -274.29723194,
#          -255.77283808,  -361.47497343,  -263.05325885,   414.25224198,
#           600.25601799,    38.63962335,   147.59972763,   373.34588176,
#           411.12032881])]
[ dl_l2(y, Xz, np.linspace(-5,5,D),0), dl_l2(y, Xz, np.linspace(-5,5,D),1) ]

In [ ]:
# now define the (f,update) tuple for optimize for logistic regression, L2
# regularization, and gradient descent
def gd_l2(y,X,lambda_):
    # YOUR CODE HERE
    def f(w):
        return -l_l2(y, X, w, lambda_)
    def update(w, eps):
        return w + eps * dl_l2(y, X, w, lambda_)
    return [f, update]

In [ ]:
# let's run!
lambda_ = 100
wz_gd_100, vz_gd_100, ez_gd_100 = optimize(gd_l2(y,Xz,lambda_), w0, nepochs=500)

lambda_ = 50
wz_gd_50, vz_gd_50, ez_gd_50 = optimize(gd_l2(y,Xz,lambda_), w0, nepochs=500)

lambda_ = 150
wz_gd_150, vz_gd_150, ez_gd_150 = optimize(gd_l2(y,Xz,lambda_), w0, nepochs=500)

## 4b Effect of Prior

In [ ]:
# Look at the training data log likelihood
print('\nTraining data log likelihood')
print('Log-Likelihood for lambda=0  ' + str(l_l2(y, Xz, np.linspace(-5,5,D), 0)))
print('Log-Likelihood for lambda=50  ' + str(l_l2(y, Xz, np.linspace(-5,5,D), 50)))
print('Log-Likelihood for lambda=100  ' + str(l_l2(y, Xz, np.linspace(-5,5,D), 100)))

# Look at the test data log likelihood
print('\nTest data log likelihood')
print('Log-Likelihood for lambda=0  ' + str(l_l2(ytest, Xtestz, np.linspace(-5,5,D), 0)))
print('Log-Likelihood for lambda=50  ' + str(l_l2(ytest, Xtestz, np.linspace(-5,5,D), 50)))
print('Log-Likelihood for lambda=100  ' + str(l_l2(ytest, Xtestz, np.linspace(-5,5,D), 100)))

# Print precision, recall and F1-Score for different configurations of lambda_
yhat = predict(Xtestz, wz_gd)
ypred = classify(Xtestz, wz_gd)
print('\nGradient descent prediction accuracy for lambda=0\n')
print( sklearn.metrics.classification_report(ytest, ypred) )

yhat_50 = predict(Xtestz, wz_gd_50)
ypred_50 = classify(Xtestz, wz_gd_50)
print('\nGradient descent prediction accuracy for lambda=50\n')
print( sklearn.metrics.classification_report(ytest, ypred_50) )

yhat_100 = predict(Xtestz, wz_gd_100)
ypred_100 = classify(Xtestz, wz_gd_100)
print('\nGradient descent prediction accuracy for lambda=100\n')
print( sklearn.metrics.classification_report(ytest, ypred_100) )

## 4c Composition of Weight Vector

In [ ]:
print('\nGradient descent statistics for lambda=0')
print(scipy.stats.describe(wz_gd))

print('\nGradient descent statistics for lambda=50')
print(scipy.stats.describe(wz_gd_50))

print('\nGradient descent statistics for lambda=100')
print(scipy.stats.describe(wz_gd_100))

print('\nGradient descent statistics for lambda=150')
print(scipy.stats.describe(wz_gd_150))

## 5 Exploration (optional)

In [ ]:
# Try gradient descent on the original data
lambda_ = 10
w_gd, v_gd, e_gd = optimize(gd_l2(y,X,lambda_), w0, nepochs=500)
yhat = predict(Xtest, w_gd)
ypred = classify(Xtest, w_gd)
print('\nGradient descent prediction accuracy for lambda=50 without z-scores\n')
print( sklearn.metrics.classification_report(ytest, ypred) )

# Leads to numerical instability due to high coefficients and high feature values
# The fallback leads for nan of inf is a label of 0.0 which results in a high precision
# for 0 values, but this is due to a larger number of non-spam e-mails in the dataset.
# This is illustrated by the recall / F1-Score that show a bad prediction for this model.

In [ ]:
# Run a logistic regression method from some existing library.

from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(Xz, y)
ypred = model.predict(Xtestz)
print(sklearn.metrics.classification_report(ytest, ypred))

# Results in a score of 0.92 which is similar to the score observed with lambda_=50